In [ ]:
from typing import Tuple
import tensorflow as tf
import numpy as np

In [ ]:
n = 10
# images is a 1 x 10 x 10 x 1 array that contains the numbers 1 through 100
images = [[[[x * n + y + 1] for y in range(n)] for x in range(n)]]

# We generate two outputs as follows:
# 1. 3x3 patches with stride length 5
# 2. Same as above, but the rate is increased to 2
tf.image.extract_patches(images=images,
                        sizes=[1, 3, 3, 1],
                        strides=[1, 5, 5, 1],
                        rates=[1, 1, 1, 1],
                        padding='VALID')

# Yields:
#[[[[ 1  2  3 11 12 13 21 22 23]
#    [ 6  7  8 16 17 18 26 27 28]]
#[[51 52 53 61 62 63 71 72 73]
#    [56 57 58 66 67 68 76 77 78]]]]

In [ ]:
tf.image.extract_patches(images=images,
                        sizes=[1, 3, 3, 1],
                        strides=[1, 5, 5, 1],
                        rates=[1, 2, 2, 1],
                        padding='VALID')

# Yields:
#[[[[  1   3   5  21  23  25  41  43  45]
#   [  6   8  10  26  28  30  46  48  50]]
#
#    [[ 51  53  55  71  73  75  91  93  95]
#     [ 56  58  60  76  78  80  96  98 100]]]]

In [ ]:
# return tensor of patches
def extract_patches(img: tf.Tensor, 
                    patch_shape: Tuple[int, int]=(3,3), 
                    strides: Tuple[int,int]=(1,1), 
                    rates: Tuple[int, int]=(1,1)) -> tf.Tensor:
    width, height = patch_shape
    stridex, stridey = strides
    ratex, ratey = rates
    ksizes = [1, width, height, 1]
    strides = [1, stridex, stridey, 1]
    rates = [1, ratex, ratey, 1]
    padding = 'VALID'  #or 'SAME'
    return tf.image.extract_patches(img, ksizes, strides, rates, padding)    

In [ ]:
x = tf.constant(4.0)
with tf.GradientTape() as tape:
  with tape.stop_recording():
    y = x ** 2
dy_dx = tape.gradient(y, x)
print(dy_dx)

In [ ]:
with tf.GradientTape() as g:
  x  = tf.constant([1.0, 2.0])
  g.watch(x)
  y = x * x
jacobian = g.jacobian(y, x)
print(jacobian)
# jacobian value is [[2., 0.], [0., 4.]]

In [ ]:
def extract_patches_inverse(img:tf.Variable, patches:tf.Variable, tape:tf.GradientTape) -> tf.Tensor:
    _x = tf.zeros_like(img)
    _y = extract_patches(_x)
    grad = tape.gradient(_y, _x)
    # Divide by grad, to "average" together the overlapping patches
    # otherwise they would simply sum up
    return (tape.gradient(_y, _x, output_gradients=patches) / grad)

In [ ]:
def extract_patches_inverse2(img, patches) -> tf.Tensor:
    _x = tf.zeros_like(img)
    _y = extract_patches(_x)
    grad = tf.gradients(_y, _x)[0]
    # Divide by grad, to "average" together the overlapping patches
    # otherwise they would simply sum up
    return (tf.gradients(_y, _x, output_gradients=patches)[0] / grad)

In [ ]:
def test_patchify():
    print('\nBegin Test')
    nx = 10
    ny = 11
    print('\nCreate 2D image')
    img = [[[[x * nx + y + 1] for y in range(ny)] for x in range(nx)]]
    print(np.asarray(img, dtype=float).squeeze())
    #img = [[[[x * ny + y + 1] for y in range(ny)] for x in range(nx)]]
    print('\nConnvert 2D image to 4D tensor')
    imgtensor = tf.Variable(tf.convert_to_tensor(img,  dtype=float))
    print('\nDisplay 2D image')
    print(tf.squeeze(imgtensor))    
    print('\ntf hack: set tape watch')
    print('\nExtract and display patches')
    with tf.GradientTape(persistent=True) as tape:
        #with tape.stop_recording():
            tape.watch(imgtensor)
            patches = extract_patches(imgtensor)
            invtensor = extract_patches_inverse(imgtensor, patches, tape)
    print(patches)
    inv = tf.squeeze(invtensor).numpy()
    print(inv)
    img = np.asarray(img, dtype=float).squeeze()
    assert img.all() == inv.all()
    print('\ntf hack: use monitored gradients to reassemble the image')
    print('\nDisplay reassembled tensor')
    print(invtensor)
    print('\nDisplay reassembled image')


In [ ]:
def test_patchify3():
    print('\nBegin Test')
    nx = 10
    ny = 11
    print('\nCreate 2D image')
    img = [[[[x * nx + y + 1] for y in range(ny)] for x in range(nx)]]
    print(np.asarray(img, dtype=float).squeeze())
    #img = [[[[x * ny + y + 1] for y in range(ny)] for x in range(nx)]]
    print('\nConnvert 2D image to 4D tensor')
    imgtensor = tf.Variable(tf.convert_to_tensor(img,  dtype=float))
    print('\nDisplay 2D image')
    print(tf.squeeze(imgtensor))    
    print('\ntf hack: set tape watch')
    print('\nExtract and display patches')
    with tf.GradientTape(persistent=True) as tape:
        #with tape.stop_recording():
            tape.watch(imgtensor)
            patches = extract_patches(imgtensor)
            invtensor = extract_patches_inverse(imgtensor, patches, tape)
    print(patches)
    inv = tf.squeeze(invtensor).numpy()
    print(inv)
    img = np.asarray(img, dtype=float).squeeze()
    assert img.all() == inv.all()
    print('\ntf hack: use monitored gradients to reassemble the image')
    print('\nDisplay reassembled tensor')
    print(invtensor)
    print('\nDisplay reassembled image')

In [ ]:
test_patchify()

In [ ]:
def test_patchify2():
    print('\nBegin Test')
    nx = 10
    ny = 11
    print('\nCreate 2D image')
    img = [[[[x * nx + y + 1] for y in range(ny)] for x in range(nx)]]
    print(np.asarray(img, dtype=float).squeeze())
    #img = [[[[x * ny + y + 1] for y in range(ny)] for x in range(nx)]]
    print('\nConnvert 2D image to 4D tensor')
    imgtensor = tf.convert_to_tensor(img,  dtype=float)
    print('\ntf hack: set tape watch')
    print('\nDisplay 2D image')
    print(tf.squeeze(imgtensor))
    print('\nExtract and display patches')
    patches = extract_patches(imgtensor)
    print(patches)
    print('\ntf hack: use monitored gradients to reassemble the image')
    invtensor = extract_patches_inverse2(imgtensor, patches)
    print('\nDisplay reassembled tensor')
    print(invtensor)
    print('\nDisplay reassembled image')
    inv = tf.squeeze(invtensor).numpy()
    print(inv)
    img = np.asarray(img, dtype=float).squeeze()
    assert img.all() == inv.all()

In [ ]:
test_patchify2()
